<a href="https://colab.research.google.com/github/Quzzal-Khanam/Hand_written_digit_recognition/blob/main/Handwritten_Digit_Recognition_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

digit_recognizer_path = kagglehub.competition_download('digit-recognizer')

print('Data source import complete.')


<p style="color:green;text-align:center;font-weight:bold;border:3px dotted green;font-size:22px">✨ Handwritten Digit Recognition with CNN✨<span style='font-size:26px; background-color:blue '></span></p>
<center><img src="https://miro.medium.com/max/1400/1*R8ZdNRRJlBs1tPkMSTwPHw.jpeg">

<h2 style="padding: 8px;color:green;border-bottom: 3px solid green"><b>1 | Introduction 🎯</b></h2>
<h3 style="color:red"><B>✅ Competition Description</b></h3>

* In this competition, our goal is to correctly identify digits from a dataset of tens of thousands of handwritten images.

<h3 style="color:red"><b>✅ Data Description</b></h3>

* The data files train.csv and test.csv contain gray-scale images of hand-drawn digits, from zero through nine.

* Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

* The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.

* Each pixel column in the training set has a name like pixelx, where x is an integer between 0 and 783, inclusive. To locate this pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27, inclusive. Then pixelx is located on row i and column j of a 28 x 28 matrix, (indexing by zero).

<h3 style="color:red"><b>✅ Model choice</b></h3>

In this kernel, I have created a model based on Deep Learning : <span style="color:green"><b>Convolutional Neural Network (CNN)</b></span> to recognize different handwritten digits and classify them. The dataset used here is actually from Digit Recognition Competition. Let's get started.

<center><img src="https://production-media.paperswithcode.com/method_collections/cnn.jpeg" style='border-radius:30px'>

<h2 style="padding: 8px;color:green;border-bottom: 3px solid green"><b>2 | Import Necessary Librairies 📚</b></h2>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import colorama
from colorama import Fore
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



<h2 style="padding: 8px;color:green;border-bottom: 3px solid green"><b>3 | Data Loading and Overview 📅</b></h2>

In [ ]:
train_df=pd.read_csv("../input/digit-recognizer/train.csv")
test_df=pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
train_df.head()

In [ ]:
print(Fore.BLUE +f"The train set has",Fore.GREEN+f"{train_df.shape[0]}",Fore.BLUE +"rows and",Fore.GREEN+f"{train_df.shape[1]}",Fore.BLUE +"columns")
print(Fore.BLUE +f"The test set has",Fore.GREEN+f"{test_df.shape[0]}",Fore.BLUE +"rows and",Fore.GREEN+f"{test_df.shape[1]}",Fore.BLUE +"columns")

In [ ]:
train_na=train_df.isna().sum().sum()
test_na=test_df.isna().sum().sum()
print(Fore.BLUE +f"The number of missing values in train set is :", Fore.GREEN+f"{train_na}")
print(Fore.BLUE +f"The number of missing values in test set is :", Fore.GREEN+f"{test_na}")

<h2 style="padding: 8px;color:green;border-bottom: 3px solid green"><b>4 | Visualization of the target variable 👁‍🗨👀</b></h2>

In [ ]:
X = train_df.drop("label", axis=1)
Y=train_df["label"]

In [ ]:
Y.value_counts()

In [ ]:
result=Y.value_counts().sort_values(ascending=False)
result.plot.barh(color=sns.color_palette("Spectral", 9),width = 0.6, figsize=(12,5))
plt.title("Number of items in each category")
plt.ylabel('Number of items')
plt.show()

<h2 style="padding: 8px;color:green;border-bottom: 3px solid green"><b>5 | Visualization of the features variables 📈📊</b></h2>

In [ ]:
X = X.values.reshape(-1,28,28,1)
X_test = test_df.values.reshape(-1,28,28,1)
X.shape, X.shape

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(13, 8))
ax[0,0].imshow(X[0], cmap = 'Greens', interpolation='bicubic')
ax[0,1].imshow(X[50], cmap = 'viridis', interpolation='bicubic')
ax[0,2].imshow(X[100], cmap = 'viridis', interpolation='bicubic')
ax[1,0].imshow(X[150], cmap = 'viridis', interpolation='bicubic')
ax[1,1].imshow(X[200], cmap = 'Greens', interpolation='bicubic')
ax[1,2].imshow(X[250], cmap = 'viridis', interpolation='bicubic')
ax[2,0].imshow(X[300], cmap = 'viridis', interpolation='bicubic')
ax[2,1].imshow(X[350], cmap = 'viridis', interpolation='bicubic')
ax[2,2].imshow(X[400], cmap = 'Greens', interpolation='bicubic')

<h2 style="padding: 8px;color:green;border-bottom: 3px solid green"><b>6 | Data preprocessing ♻</b></h2>

In [ ]:
# Data normalization
X = X/255
X_test = X_test/255

In [ ]:
# one-hot encode target column
Y = to_categorical(Y, num_classes = 10)

In [ ]:
# Spliting Data in training and testing set
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.3, random_state=42)

<h2 style="padding: 8px;color:green;border-bottom: 3px solid green"><b>7 | Building a CNN model ⚔</b></h2>

In [ ]:
#create model
model = models.Sequential()
# Add layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

In [ ]:
model.summary()

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# Visualization of the model
tf.keras.utils.plot_model(model)

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=10,
                    validation_data=(X_val, Y_val))

<h2 style="padding: 8px;color:green;border-bottom: 3px solid green"><b>8 | Model evaluation & accuracy</b></h2>

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
train_loss, train_accuracy = model.evaluate(X_train, Y_train)
print('Train loss: ', train_loss)
print('Train accuracy: ', train_accuracy)

In [ ]:
validation_loss, validation_accuracy = model.evaluate(X_val, Y_val)
print('Validation loss: ', validation_loss)
print('Validation accuracy: ', validation_accuracy)

<h2 style="padding: 8px;color:green;border-bottom: 3px solid green"><b>9 | Prediction & submission</b></h2>

In [ ]:
# predict results
results = model.predict(X_test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submission.csv",index=False)